In [5]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox
from pyro import clear_param_store
import pyro as pyro
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import torch
import kennard_stone as ks
import arviz as az
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral
from bokeh.models import Slope, Div
from bokeh.io import curdoc
from bokeh.layouts import column,gridplot
from print_versions import print_versions
from sklearn.metrics import r2_score
from seaborn import clustermap
from bokeh.models import Band, ColumnDataSource
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [6]:
print_versions(globals());

json==2.0.9
pandas==2.2.3
ipykernel==6.28.0
numpy==1.26.4
tensorflow==2.18.0
keras==3.5.0
torch==2.5.1
ipywidgets==7.6.5
xarray==2023.6.0
osmnx==1.9.3
pyro==1.9.1
kennard_stone==3.0.0
arviz==0.20.0
scipy==1.14.0
bokeh==3.6.0
seaborn==0.13.2


In [7]:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,examine,help"

In [8]:
PM25=pd.read_excel(r"corectiong_123.xlsx");
PM25_raw=pd.read_excel(r"pm25.xlsx");


In [9]:
PM25=pd.concat([PM25_raw,PM25.loc[:,['constrution_year', 'Area', 'floor_level', 'powerplant', 'road', 'motorway', 'forest', 'coastline',
       'airport', 'farmland', 'industrial', 'carpet', 'cookerhood', 'passive',
       'gastove', 'fireplace']]],axis=1);
PM25=PM25.drop(['Unnamed: 0', 'fnr',"adrnum",'vejnvn', 'husn',"postnr"],axis=1);

In [10]:
PM25_flow=PM25.drop(['sadto'],axis=1)

In [11]:
PM25.isna().sum(axis=1)

0       0
1       0
2       0
3       1
4       1
       ..
1117    0
1118    0
1119    1
1120    0
1121    0
Length: 1122, dtype: int64

In [12]:
PM25=PM25_flow.loc[PM25_flow.passive.isna()==0,:];
scaler = StandardScaler()
imputer = KNNImputer(n_neighbors=10)
pm25=imputer.fit_transform(scaler.fit_transform(PM25))
pm25=scaler.inverse_transform(pm25)

In [13]:
PM25

,bnr,flow,pm_final,sv_final,maalnr,constrution_year,Area,floor_level,powerplant,road,...,forest,coastline,airport,farmland,industrial,carpet,cookerhood,passive,gastove,fireplace
0,1,4.07,3.747515,0.162272,1,1979.0,126.0,0,0.000000,11222.957551,...,1086.639874,0.000000,7677.52143,767180.281538,0.000000,0.0,365.0,365.0,365.0,0.0
1,1,3.75,13.850762,3.059650,2,1979.0,126.0,0,0.000000,11222.957551,...,1086.639874,0.000000,7677.52143,767180.281538,0.000000,0.0,365.0,365.0,365.0,1.0
2,1,4.22,6.937794,0.342990,3,1979.0,126.0,0,0.000000,11222.957551,...,1086.639874,0.000000,7677.52143,767180.281538,0.000000,0.0,365.0,365.0,365.0,1.0
5,5,4.11,6.209266,0.417824,1,1997.0,134.0,0,0.000000,10507.822015,...,16507.818218,0.000000,0.00000,687250.510566,0.000000,365.0,365.0,0.0,0.0,1.0
6,5,3.84,13.870286,1.940405,2,1997.0,134.0,0,0.000000,10507.822015,...,16507.818218,0.000000,0.00000,687250.510566,0.000000,365.0,365.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,450,3.94,18.170482,1.199333,2,1968.0,94.0,1,0.000000,20693.860407,...,3691.099407,0.000000,0.00000,58691.722066,2386.805052,0.0,104.0,730.0,0.0,0.0
1117,450,3.96,13.039686,0.562392,3,1968.0,94.0,1,0.000000,20693.860407,...,3691.099407,0.000000,0.00000,58691.722066,2386.805052,0.0,104.0,730.0,0.0,0.0
1118,451,3.88,23.291217,1.544408,1,1987.0,95.0,0,0.000000,20693.860407,...,3691.099407,0.000000,0.00000,58691.722066,2386.805052,0.0,365.0,0.0,180.0,100.0
1120,452,4.05,19.422368,2.665289,2,1925.0,79.0,1,318.093389,21417.657017,...,275.455935,1053.572362,0.00000,265.889267,0.000000,0.0,0.0,60.0,365.0,3.0


In [14]:
PM25=pd.DataFrame(np.round(pm25,1),columns=PM25.columns,index=PM25.index)

In [15]:
bnr=PM25.loc[:,["bnr","Area","passive","flow"]].groupby("bnr").mean().reset_index()
X_train, X_test, y_train, y_test = ks.train_test_split(bnr.loc[:,["Area","passive","flow"]], bnr.bnr, test_size = 0.2);
PM25=PM25.set_index("bnr")
train=PM25.loc[y_train.values,:]
test=PM25.loc[y_test.values,:]


2025-02-07 13:32:20,173 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.

2025-02-07 13:32:20,176 - kennard_stone.utils._pairwise:109[INFO] - Calculating pairwise distances using scikit-learn.



In [17]:
traning_pm25 = {"X":torch.tensor(train.flow.values).float(),"y":torch.tensor(train.pm_final.values).float()};
traning_bc = {"X":torch.tensor(train.flow.values).float(),"y":torch.tensor(train.sv_final.values).float()};
test_bc = {"X":torch.tensor(test.flow.values).float(),"y":torch.tensor(test.sv_final.values).float()};
test_pm25 = {"X":torch.tensor(test.flow.values).float(),"y":torch.tensor(test.pm_final.values).float()};

In [18]:
clear_param_store();
rbf = gp.kernels.RBF(input_dim=1);

rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(10.)));

rbf.lengthscale = PyroSample(dist.LogNormal(torch.tensor(0.), torch.tensor(0.4)));
gpr = gp.models.GPRegression(traning_pm25["X"],traning_pm25["y"], rbf);
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(20.)));

nuts_kernel = NUTS(gpr.model,target_accept_prob=0.9);

mcmc = MCMC(nuts_kernel, num_samples=1000,warmup_steps=3000,num_chains=1);

mcmc.run();

Sample: 100%|███████████████████████████████████████| 4000/4000 [09:38,  6.91it/s, step size=7.03e-01, acc. prob=0.936]


In [19]:
mcmc.diagnostics();

In [20]:
posterior_samples = mcmc.get_samples();
posterior_predictive = Predictive(gpr, posterior_samples)(traning_pm25["X"]);
prior = Predictive(gpr, num_samples=500)(traning_pm25["X"]);

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive

);
pyro_data;

C:\Users\michael.forsmann\anaconda3\Lib\site-packages\arviz\data\io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


In [21]:
az.style.use("arviz-doc");

ax = az.plot_trace(pyro_data,backend='bokeh');

In [22]:
ax = az.plot_density(
    [pyro_data],
    shade=0.1,backend='bokeh'
);

In [23]:
linmod=torch.linspace(3.3,4.7,800)
pm25_model,pm25_model_std=gpr(linmod,full_cov=True)

linmod_np=linmod.detach().numpy()
pm25_model_np,pm25_model_std_np=pm25_model.detach().numpy().copy(),pm25_model_std.diag().sqrt().detach().numpy().copy()


lower1 = pm25_model_np - pm25_model_std_np
upper1 = pm25_model_np + pm25_model_std_np



data1=pd.DataFrame([linmod_np,lower1,upper1],index=["flow","lower","upper"]).T
data1 = ColumnDataSource(data1.reset_index())

In [24]:
torch.save(gpr, "PM2.5_flow_correction");

In [25]:
exp_curve=curve_fit(lambda t,a,b: a*np.exp(b*t),  train.flow,train.pm_final);
linear_curve=curve_fit(lambda t,a,b: a * t +b,  train.flow,train.pm_final);
log_curve=curve_fit(lambda t,a,b: a * t +b,  train.flow,np.log(train.pm_final));

In [26]:
def exp(x, a, b):
    return a * np.exp(b * x);
def linear(x, a, b):
    return a * x +b;


In [27]:
x=np.linspace(2,5,200)
X_tor=torch.tensor(x).float()
data=pd.DataFrame(x,index=np.arange(200),columns=["flow"])
data["lieanr_scale_pm"]=(linear(4,*linear_curve[0])/linear(x,*linear_curve[0]))
data["exp_scale_pm"]=exp(4,*exp_curve[0])/exp(x,*exp_curve[0])
data["guassion_scale_pm"]=(gpr(torch.tensor(4*np.ones(len(x))).float())[0]/gpr(X_tor)[0]).detach().numpy()
data["log_scale_pm"]=linear(4,*log_curve[0])/linear(x,*log_curve[0])

In [28]:
r2_score(test.pm_final,linear(test.flow,*linear_curve[0])),r2_score(test.pm_final,gpr(torch.tensor(test.flow.values).float())[0].detach().numpy()),r2_score(test.pm_final,exp(test.flow,*exp_curve[0])),r2_score(test.pm_final,linear(test.flow,*linear_curve[0]))

(-0.12828858018130185,
 -0.11264449098064766,
 -0.12162450562557581,
 -0.12828858018130185)

In [29]:
def rmse(y_raw,y_predict):
    return np.sqrt(np.mean(((y_raw-y_predict)**2)))

In [30]:
rmse(test.pm_final,linear(test.flow,*linear_curve[0])),rmse(test.pm_final,gpr(torch.tensor(test.flow.values).float())[0].detach().numpy()),rmse(test.pm_final,exp(test.flow,*exp_curve[0])),rmse(test.pm_final,linear(test.flow,*linear_curve[0]))

(14.241742550798444,
 14.142664712380574,
 14.199621859605191,
 14.241742550798444)

In [31]:
clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)

rbf_bc.variance = PyroSample(dist.HalfNormal(torch.tensor(5.)))

rbf_bc.lengthscale = PyroSample(dist.LogNormal(torch.tensor(0.), torch.tensor(0.4)))
gpr_bc = gp.models.GPRegression(traning_bc["X"],traning_bc["y"], rbf)
gpr_bc.noise = PyroSample(dist.HalfNormal(torch.tensor(5.)))


nuts_kernel_bc = NUTS(gpr_bc.model)


mcmc_bc = MCMC(nuts_kernel_bc,warmup_steps=2000, num_samples=1000)

mcmc_bc.run()

Sample: 100%|███████████████████████████████████████| 3000/3000 [07:41,  6.50it/s, step size=8.03e-01, acc. prob=0.875]


In [32]:
Predictive(gpr_bc, posterior_samples_bc)(traning_bc["X"])

NameError: name 'posterior_samples_bc' is not defined

In [ ]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(traning_bc["X"])
prior_bc = Predictive(gpr_bc, num_samples=500)(traning_bc["X"])

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
pyro_data

In [ ]:
az.style.use("arviz-doc")

ax = az.plot_trace(pyro_data_bc,backend='bokeh')

In [ ]:
ax = az.plot_density(
    [pyro_data_bc],
    shade=0.1,backend='bokeh'
)

In [ ]:
az.plot_pair(centered,kind='kde',divergences=True,textsize=18)

In [ ]:
torch.save(gpr_bc, "BC_flow_correction")





In [ ]:
exp_curve_bc=curve_fit(lambda t,a,b: a*np.exp(b*t),  train.flow,train.sv_final)
linear_curve_bc=curve_fit(lambda t,a,b: a * t +b,  train.flow,train.sv_final)
log_curve_bc=curve_fit(lambda t,a,b: a * t +b,  train.flow,np.log(train.sv_final))

In [ ]:
X=torch.tensor(train.flow.values).float()
y=torch.tensor(train.sv_final.values).float()
y_pm=torch.tensor(train.pm_final.values).float()

In [ ]:
train["PM25_Gaussion"]=(y_pm*gpr(torch.tensor(4*np.ones(len(X))).float())[0]/gpr(X)[0]).detach().numpy()
train["pm25_linear"]=linear(4,*linear_curve[0])/linear(train.flow.values,*linear_curve[0])*train.pm_final.values
train["pm25_exp"]=exp(4,*exp_curve[0])/exp(train.flow,*exp_curve[0])*train.pm_final.values
train["pm25_log"]=linear(4,*log_curve[0])/linear(train.flow,*log_curve[0])*train.pm_final.values

In [ ]:
train["bc_Gaussion"]=(y*gpr_bc(torch.tensor(4*np.ones(len(X))).float())[0]/gpr_bc(X)[0]).detach().numpy()
train["bc_linear"]=linear(4,*linear_curve_bc[0])/linear(train.flow,*linear_curve_bc[0])*train.sv_final
train["bc_exp"]=exp(4,*exp_curve_bc[0])/exp(train.flow,*exp_curve_bc[0])*train.sv_final
train["bc_log"]=linear(4,*log_curve_bc[0])/linear(train.flow,*log_curve_bc[0])*train.sv_final

In [ ]:
X=torch.tensor(PM25.flow.values).float()
y=torch.tensor(PM25.sv_final.values).float()
y_pm=torch.tensor(PM25.pm_final.values).float()

In [ ]:
PM25["BC_Gaussion"]=(y *gpr_bc(torch.tensor(4*np.ones(len(PM25.flow))).float())[0]/gpr_bc(X)[0]).detach().numpy()
PM25["BC_linear"]=linear(4,*linear_curve_bc[0])/linear(PM25.flow,*linear_curve_bc[0])*PM25.sv_final
PM25["BC_exp"]=exp(4,*exp_curve_bc[0])/exp(PM25.flow,*exp_curve_bc[0])*PM25.sv_final
PM25["BC_log"]=linear(4,*log_curve_bc[0])/linear(PM25.flow,*log_curve_bc[0])*PM25.sv_final

In [ ]:
PM25["pm25_Gaussion"]=(y_pm *gpr(torch.tensor(4*np.ones(len(PM25.flow))).float())[0]/gpr(X)[0]).detach().numpy()
PM25["pm25_linear"]=linear(4,*linear_curve[0])/linear(PM25.flow,*linear_curve[0])*PM25.pm_final
PM25["pm25_exp"]=exp(4,*exp_curve[0])/exp(PM25.flow,*exp_curve[0])*PM25.pm_final
PM25["pm25_log"]=linear(4,*log_curve[0])/linear(PM25.flow,*log_curve[0])*PM25.pm_final

In [ ]:


bc_model,bc_model_std=gpr_bc(linmod, full_cov=True)
bc_model_np,bc_model_std_np=bc_model.detach().numpy().copy(),bc_model_std.diag().sqrt().detach().numpy().copy()
lower1_bc = bc_model_np - bc_model_std_np
upper1_bc = bc_model_np + bc_model_std_np
data1_bc=pd.DataFrame([linmod_np,lower1_bc,upper1_bc],index=["flow","lower_bc","upper_bc"]).T
data1_bc = ColumnDataSource(data1_bc.reset_index())

In [ ]:
bc_model_std_np,pm_model_std_np

In [ ]:
std=train.sort_values("flow").rolling(50, min_periods=1,closed='both',step=1).std()
mean=train.sort_values("flow").rolling(50, min_periods=1,closed='both',step=1).mean()
std.columns=std.columns+"_std"
mean=pd.concat([mean,std],axis=1)
lower1_std = mean.pm_final - mean.pm_final_std
upper1_std = mean.pm_final + mean.pm_final_std
lower1_bc_std  = mean.sv_final - mean.sv_final_std
upper1_bc_std  = mean.sv_final + mean.sv_final_std
mean["lower1_std_pm"]=lower1_std
mean["upper1_std_pm"]=upper1_std
mean["lower1_std_bc"]=lower1_bc_std
mean["upper1_std_bc"]=upper1_bc_std

mean=mean.dropna()
mean1 = ColumnDataSource(mean.reset_index())

In [ ]:
train

In [ ]:
p = figure(tools=TOOLS);

p.xgrid.grid_line_color=None;
p.ygrid.grid_line_alpha=0.5;
p.line(mean.flow, mean.pm_final, line_width=3,color="green",legend_label="mean_roll=100");
p.line(linmod_np, pm25_model_np, line_width=3,color="red",legend_label="Gaussion model");
p.scatter(train.flow, y=train.pm_final, color="blue", marker="dot", size=20, alpha=0.4,legend_label="raw points");
band = Band(base="flow", lower="lower1_std_pm", upper="upper1_std_pm",source=mean1, fill_color="red", line_color="black");
band1 = Band(base="flow", lower="lower", upper="upper",source=data1,fill_alpha=0.5, fill_color="blue", line_color="black");

p.add_layout(band);
p.add_layout(band1);

p1 = figure(tools="", toolbar_location=None);

p1.xgrid.grid_line_color=None;
p1.ygrid.grid_line_alpha=0.5;
p1.line(mean.flow, mean.sv_final, line_width=3,color="green",legend_label="mean_roll=100");
p1.line(linmod_np, bc_model_np, line_width=3,color="red",legend_label="Gaussion model");
p1.scatter(train.flow, y=train.sv_final, color="blue", marker="dot", size=20, alpha=0.4,legend_label="raw points");
band2 = Band(base="flow", lower="lower_bc", upper="upper_bc",source=data1_bc, fill_color="blue", line_color="black");
band3 = Band(base="flow", lower="lower1_std_bc", upper="upper1_std_bc",source=mean1,fill_alpha=0.5, fill_color="red", line_color="black");
p1.add_layout(band3);
p1.add_layout(band2);
p.title.text = r"$$ PM_{2.5}$$ calibration of flow";
p1.title.text = r" Black Carbon calibration of flow";
p1.xaxis.axis_label = r'$$Flow$$';
p1.yaxis.axis_label = 'Black carbon ';
p.xaxis.axis_label = r'$$Flow$$';
p.yaxis.axis_label = r'$$PM_{2.5}$$ ';
p1.legend.title_text_font_size = "24px";
p.legend.title_text_font_size = "24px";

p.xaxis.axis_label_text_font_size = "20px";
p.yaxis.axis_label_text_font_size = "20px";
p1.xaxis.axis_label_text_font_size = "20px";
p1.yaxis.axis_label_text_font_size = "20px";

p1.yaxis.axis_label_text_font_style = "bold";
p.xaxis.axis_label_text_font_style = "bold";

p.yaxis.axis_label_text_font_style = "bold";
p1.xaxis.axis_label_text_font_style = "bold";
grid =  gridplot([[p, p1]], width=500, height=500);

show(column(grid));

In [ ]:
p1=np.polyfit(PM25.flow.values,PM25.pm_final,1);
p2=np.polyfit(PM25.flow.values,PM25.pm25_Gaussion,1);
p3=np.polyfit(PM25.flow.values,PM25.pm25_linear,1);
p4=np.polyfit(PM25.flow.values,PM25.pm25_exp,1);

In [ ]:
x=np.linspace(3.4,4.6,100)

In [ ]:

p = figure(tools="", toolbar_location=None)
p.scatter(PM25.flow, y=PM25.pm_final, color="blue", marker="dot", size=20, alpha=0.4);
p.line(x, linear(x,*p1), line_width=3,legend_label="Raw fit", line_color="red");
p.line(x, linear(x,*p2), line_width=3,legend_label="Gaussion correction", line_color="green");
p.line(x, linear(x,*p3), line_width=3,legend_label="Linaer correction", line_color="yellow");
p.line(x, linear(x,*p4), line_width=3,legend_label="Exponential correction", line_color="brown");

show(p)

In [ ]:
r2_score(test.sv_final,linear(test.flow,*linear_curve_bc[0])),r2_score(test.sv_final.values,gpr_bc(torch.tensor(test.flow.values).float())[0].detach().numpy()),r2_score(test.sv_final,exp(test.flow,*exp_curve_bc[0])),r2_score(test.sv_final,linear(test.flow,*linear_curve_bc[0]))

In [ ]:
rmse(test.sv_final,linear(test.flow,*linear_curve_bc[0])),rmse(test.sv_final,gpr_bc(torch.tensor(test.flow.values).float())[0].detach().numpy()),rmse(test.sv_final,exp(test.flow,*exp_curve_bc[0])),rmse(test.sv_final,linear(test.flow,*linear_curve_bc[0]))

In [ ]:
x=np.linspace(3.4,4.6,200)
x_tor=torch.tensor(x).float()

In [ ]:

data["lieanr_scale_pm"]=(linear(4,*linear_curve[0])/linear(x,*linear_curve[0]))
data["exp_scale_pm"]=exp(4,*exp_curve[0])/exp(x,*exp_curve[0])
data["guassion_scale_pm"]=(gpr(torch.tensor(4*np.ones(len(X_tor))).float())[0]/gpr(X_tor)[0]).detach().numpy()
data["log_scale_pm"]=linear(4,*log_curve[0])/linear(x,*log_curve[0])

In [ ]:

data["lieanr_scale_bc"]=(linear(4,*linear_curve_bc[0])/linear(x,*linear_curve_bc[0]))
data["exp_scale_bc"]=exp(4,*exp_curve_bc[0])/exp(x,*exp_curve_bc[0])
data["guassion_scale_bc"]=(gpr_bc(torch.tensor(4*np.ones(len(X_tor))).float())[0]/gpr_bc(X_tor)[0]).detach().numpy()
data["log_scale_bc"]=linear(4,*log_curve_bc[0])/linear(x,*log_curve_bc[0])

In [ ]:
p = figure(title = "Penguin size", background_fill_color="#fafafa",tools=TOOLS)

p.line(data.guassion_scale_pm, data.guassion_scale_bc, line_color="blue",
        line_width=3, legend_label="Linear")
show(p)

In [ ]:
p = figure(title = r"$$PM_{2.5}$$", background_fill_color="#fafafa",tools=TOOLS)

p.line(data.flow, data.lieanr_scale_pm, line_color="blue", line_width=3, legend_label="Linear")
p.line(data.flow, data.exp_scale_pm, line_color="red", line_width=3, legend_label="Exponential")
p.line(data.flow, data.guassion_scale_pm, line_color="green",line_width=3, legend_label="Gaussian")
p.line(data.flow, data.log_scale_pm, line_color="yellow", line_width=3, legend_label="Log")
p.legend.location = "top_left"
p.legend.title = r"Correction"
p.legend.title_text_font_size = "16px"
p.legend.title_text_font_style = "bold"
p.xaxis.axis_label = r'$$Flow \frac{L}{s}$$'
p.yaxis.axis_label = 'scale'
p.xaxis.axis_label_text_font_size = "14px"
p.yaxis.axis_label_text_font_size = "16px"
p.yaxis.axis_label_text_font_style = "bold"
p.xaxis.axis_label_text_font_style = "bold"

p1 = figure(title = "Black Carbon", background_fill_color="#fafafa")

p1.line(data.flow, data.lieanr_scale_bc, line_color="blue",
        line_width=3, legend_label="Linear")
p1.line(data.flow, data.exp_scale_bc, line_color="red", line_width=3, legend_label="Exponential")
p1.line(data.flow, data.guassion_scale_bc, line_color="green", line_width=3, legend_label="Gaussian")
#p1.line(data.flow, data.log_scale_bc, line_color="yellow", line_width=3, legend_label="Log")
p1.legend.location = "top_left"
p1.legend.title = "Correction"
p1.legend.title_text_font_size = "16px"
p1.xaxis.axis_label_text_font_size = "14px"
p1.yaxis.axis_label_text_font_size = "16px"
p1.yaxis.axis_label_text_font_style = "bold"
p1.xaxis.axis_label_text_font_style = "bold"

p1.legend.title_text_font_style = "bold"
p1.xaxis.axis_label = r'$$Flow \frac{L}{s}$$'
p1.yaxis.axis_label = 'scale'
grid =  gridplot([[p, p1]], width=500, height=500)

show(column(grid))

In [3]:
PM25

NameError: name 'PM25' is not defined

In [435]:
PM25.to_excel("pm25_BC_corrected.xlsx")